In [ ]:
# Reference code from Session 13 - Heaps and priority queues

import heapq
import math

class Task:
    """
    - id: Task id (a reference number)   
    - description: Task short description   
    - duration: Task duration in minutes   
    - dependencies: List of task ids that need to preceed this task
    - earliest_start_time: The earliest time (in minutes from midnight) the task can start
    - status: Current status of the task     
   
    """
    # Initializes an instance of Task
    def __init__(self, id, description, duration,
                 dependencies, earliest_start_time=None, status="N"):
        self.id = id
        self.description = description
        self.duration = duration
        self.dependencies = dependencies
        self.earliest_start_time = earliest_start_time
        self.status = status

    def __lt__(self, other):
        return self.id < other.id    
    
class TaskScheduler:
    """
    An Advanced Daily Task Scheduler Using Priority Queues
    that now respects time constraints.
    """
    NOT_STARTED = 'N'
    IN_PRIORITY_QUEUE = 'I'
    COMPLETED = 'C'
    
    def __init__(self, tasks):
        self.tasks = tasks
        # --- FIX ---
        # Create a lookup map for task IDs to task objects
        # This allows us to find task descriptions from just an ID
        self.task_map = {task.id: task for task in tasks} 
        # --- END FIX ---
        
        self.priority_queue = []
        self.time_blocked_tasks = []
        
    def print_self(self):
        print("Tasks added to the advanced scheduler:")
        print("---------------------------------------")
        for t in self.tasks:
            print(f"➡️'{t.description}', duration = {t.duration} mins.")   
            
            if t.dependencies:
                # Look up the description for each dependency ID
                try:
                    dep_descriptions = [self.task_map[dep_id].description for dep_id in t.dependencies]
                    print(f"\t ⚠️ Depends on: \"{', '.join(dep_descriptions)}\"")
                except KeyError as e:
                    print(f"\t ⚠️ Error: Dependency ID {e} not found!")
            # --- END FIX ---
            
            if t.earliest_start_time:
                print(f"\t ⏰ Cannot start before: {self.format_time(t.earliest_start_time)}")
            
    def remove_dependency(self, id):
        for t in self.tasks:
            if t.id != id and id in t.dependencies:
                t.dependencies.remove(id)           
            
    def get_tasks_ready(self):
        """Finds tasks with no dependencies and moves them to the queue."""
        for task in self.tasks:
            if task.status == self.NOT_STARTED and not task.dependencies: 
                task.status = self.IN_PRIORITY_QUEUE 
                heapq.heappush(self.priority_queue, task)
    
    def move_ready_time_blocked_tasks(self, current_time):
        """
        Moves tasks from the time-blocked list back to the priority queue
        if their start time has been reached.
        """
        tasks_to_move = []
        for task in self.time_blocked_tasks:
            # Check if task is not None and its start time is met
            if task and task.earliest_start_time is not None and current_time >= task.earliest_start_time:
                tasks_to_move.append(task)
        
        for task in tasks_to_move:
            self.time_blocked_tasks.remove(task)
            heapq.heappush(self.priority_queue, task)

    def check_unscheduled_tasks(self):
        """Checks if any tasks are still not completed."""
        for task in self.tasks:
            if task.status != self.COMPLETED:
                return True
        return False   
    
    def format_time(self, time):
        # Handle potential None or non-numeric time values gracefully
        if not isinstance(time, (int, float)):
            return "Invalid time"
        return f"{int(time)//60}h{int(time)%60:02d}"
    
    def run_task_scheduler(self, starting_time):
        current_time = starting_time
        print("Running an advanced scheduler:\n")
        
        while self.check_unscheduled_tasks():
            
            self.get_tasks_ready()
            self.move_ready_time_blocked_tasks(current_time)
            
            if self.priority_queue:
                task = heapq.heappop(self.priority_queue)
                
                if task.earliest_start_time is None or current_time >= task.earliest_start_time:
                    print(f"🕰t={self.format_time(current_time)}")
                    print(f"\tstarted '{task.description}' for {task.duration} mins...")
                    current_time += task.duration            
                    print(f"\t✅ t={self.format_time(current_time)}, task completed!") 
                    self.remove_dependency(task.id)
                    task.status = self.COMPLETED
                else:
                    self.time_blocked_tasks.append(task)
            
            elif self.time_blocked_tasks:
                # Filter out None values before finding the minimum
                valid_times = [t.earliest_start_time for t in self.time_blocked_tasks if t.earliest_start_time is not None]
                if not valid_times:
                    # Should not happen if tasks are added correctly, but good to check
                    print(f"🕰t={self.format_time(current_time)} - ERROR: Time-blocked tasks have no start time!")
                    break 
                
                next_available_time = min(valid_times)
                
                if current_time < next_available_time:
                    print(f"🕰t={self.format_time(current_time)}... no tasks ready. Waiting until {self.format_time(next_available_time)}...")
                    current_time = next_available_time
            
            elif self.check_unscheduled_tasks():
                 print(f"🕰t={self.format_time(current_time)} - ERROR: Deadlock detected!")
                 print("The following tasks are un-runnable (check for dependency cycles):")
                 for t in self.tasks:
                     if t.status == self.NOT_STARTED:
                         dep_descs = [self.task_map[dep_id].description for dep_id in t.dependencies]
                         print(f" - {t.description}, dependencies: \"{', '.join(dep_descs)}\"")
                 break
            
        total_time = current_time - starting_time             
        print(f"\n🏁 Completed all planned tasks in {int(total_time)//60}h{int(total_time)%60:02d}min!")

In [46]:
# Reference code from Session 13 - Heaps and priority queues

"""
List of Tasks - 5 Nov 2025:

• Non-timely tasks:
1) Make CS113 Pre-Class Work
2) Have Japan's signature drink - Matcha Green Tea
3) Go to the gym
4) Buy 100¥ (~$0.65) Snacks for Kamaishi Trip at Mini Stop after Lincos
5) Take shower after gym
6) Have dinner

• Timely Tasks
1) Wake up at 09:30
2) Have lunch at 14:30
3) Buy Bento at Lincos after gym after 20:00
4) Go to bed before 02:30

"""
tasks = [
    Task(id=0, description='Wake up at 09:30', 
         duration=10, dependencies=[], earliest_start_time=570), 
    Task(id=1, description='Have lunch at 14:30', 
         duration=60, dependencies=[0], earliest_start_time=870), 
    Task(id=2, description='Go to the gym', 
         duration=120, dependencies=[0, 1]), 
    Task(id=3, description='Buy Bento at Lincos after gym after 20:00', 
         duration=25, dependencies=[0, 2], earliest_start_time=1200), 
    Task(id=4, description='Buy 100¥ (~$0.65) Snacks for Kamaishi Trip at Mini Stop after Lincos', 
         duration=10, dependencies=[0, 1, 3]), 
    Task(id=5, description='Take shower after gym', 
         duration=30, dependencies=[0, 2]), 
    Task(id=6, description='Have dinner', 
         duration=40, dependencies=[0, 5]), 
    Task(id=7, description='Prepare luggage for Kamaishi Trip', 
         duration=100, dependencies=[0, 6]),
     Task(id=8, description='Make CS113 Pre-Class Work', 
         duration=120, dependencies=[0, 7]),
     Task(id=9, description="Have Japan's signature drink - Matcha Green Tea", 
         duration=15, dependencies=[0]),
     Task(id=10, description='Go to bed before 02:30', 
         duration=15, dependencies=[0, 1, 6, 7, 8])
    ]

task_scheduler = TaskScheduler(tasks)

# print the scheduler's input 
task_scheduler.print_self()

Tasks added to the advanced scheduler:
---------------------------------------
➡️'Wake up at 09:30', duration = 10 mins.
	 ⏰ Cannot start before: 9h30
➡️'Have lunch at 14:30', duration = 60 mins.
	 ⚠️ Depends on: "Wake up at 09:30"
	 ⏰ Cannot start before: 14h30
➡️'Go to the gym', duration = 120 mins.
	 ⚠️ Depends on: "Wake up at 09:30, Have lunch at 14:30"
➡️'Buy Bento at Lincos after gym after 20:00', duration = 25 mins.
	 ⚠️ Depends on: "Wake up at 09:30, Go to the gym"
	 ⏰ Cannot start before: 20h00
➡️'Buy 100¥ (~$0.65) Snacks for Kamaishi Trip at Mini Stop after Lincos', duration = 10 mins.
	 ⚠️ Depends on: "Wake up at 09:30, Have lunch at 14:30, Buy Bento at Lincos after gym after 20:00"
➡️'Take shower after gym', duration = 30 mins.
	 ⚠️ Depends on: "Wake up at 09:30, Go to the gym"
➡️'Have dinner', duration = 40 mins.
	 ⚠️ Depends on: "Wake up at 09:30, Take shower after gym"
➡️'Prepare luggage for Kamaishi Trip', duration = 100 mins.
	 ⚠️ Depends on: "Wake up at 09:30, Have di

In [47]:
# Reference code from Session 13 - Heaps and priority queues
start_scheduler_at = int((9.5)*60)
task_scheduler.run_task_scheduler(start_scheduler_at)

Running an advanced scheduler:

🕰t=9h30
	started 'Wake up at 09:30' for 10 mins...
	✅ t=9h40, task completed!
🕰t=9h40
	started 'Have Japan's signature drink - Matcha Green Tea' for 15 mins...
	✅ t=9h55, task completed!
🕰t=9h55... no tasks ready. Waiting until 14h30...
🕰t=14h30
	started 'Have lunch at 14:30' for 60 mins...
	✅ t=15h30, task completed!
🕰t=15h30
	started 'Go to the gym' for 120 mins...
	✅ t=17h30, task completed!
🕰t=17h30
	started 'Take shower after gym' for 30 mins...
	✅ t=18h00, task completed!
🕰t=18h00
	started 'Have dinner' for 40 mins...
	✅ t=18h40, task completed!
🕰t=18h40
	started 'Prepare luggage for Kamaishi Trip' for 100 mins...
	✅ t=20h20, task completed!
🕰t=20h20
	started 'Buy Bento at Lincos after gym after 20:00' for 25 mins...
	✅ t=20h45, task completed!
🕰t=20h45
	started 'Buy 100¥ (~$0.65) Snacks for Kamaishi Trip at Mini Stop after Lincos' for 10 mins...
	✅ t=20h55, task completed!
🕰t=20h55
	started 'Make CS113 Pre-Class Work' for 120 mins...
	✅ t=22h55, t